<a href="https://colab.research.google.com/github/harveenchadha/bol/blob/main/demos/gr/hindi/Speech_ASR_plus_Translation_Hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%%capture
!pip install gradio
!pip install transformers torchaudio
!apt-get install sox
!pip install sentencepiece

In [7]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model_translate = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer_translate = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
inlang='hi'
outlang='en'
tokenizer_translate.src_lang = inlang

def translate(text):    

    encoded_hi = tokenizer_translate(text, return_tensors="pt")
    generated_tokens = model_translate.generate(**encoded_hi, forced_bos_token_id=tokenizer_translate.get_lang_id(outlang))
    return tokenizer_translate.batch_decode(generated_tokens, skip_special_tokens=True)[0]


In [8]:
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import argparse
from glob import glob
import torchaudio
import subprocess
import gradio as gr

resampler = torchaudio.transforms.Resample(48_000, 16_000)
processor = Wav2Vec2Processor.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")
model = Wav2Vec2ForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")


def get_filename(wav_file):
    filename_local = wav_file.split('/')[-1][:-4]
    filename_new = '/tmp/'+filename_local+'_16.wav'    
    subprocess.call(["sox {} -r {} -b 16 -c 1 {}".format(wav_file, str(16000), filename_new)], shell=True)
    return filename_new

def parse_transcription(wav_file):
    # load pretrained model
    
    # load audio

    wav_file = get_filename(wav_file.name)
    audio_input, sample_rate = sf.read(wav_file)
    #test_file = resampler(test_file[0])

    # pad input values and return pt tensor
    input_values = processor(audio_input, sampling_rate=16_000, return_tensors="pt").input_values

    # INFERENCE
    # retrieve logits & take argmax
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # transcribe
    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription, translate(transcription)
    #return transcription

In [9]:
from gradio.mix import Parallel, Series

In [10]:
input = gr.inputs.Audio(source="microphone", type="file", label="Please speak into the microphone") 
output1 = gr.outputs.Textbox(label="Hindi Output from ASR")
output2 = gr.outputs.Textbox(label="English Translated Output")


gr.Interface(parse_transcription, inputs = input,  outputs=[output1, output2], analytics_enabled=False, 
                                                                            show_tips=False, 
                                                                            theme='huggingface',
                                                                            layout='vertical',
                                                                            title="Vakyansh: Speech To text for Indic Languages",
                                                                            description="This is a live demo for Speech to Text and Speech Translation. <br> Models used: wav2vec2 + m2m100").launch( inline=False)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Your interface requires microphone or webcam permissions - this may cause issues in Colab. Use the External URL in case of issues.
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://56367.gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7861/',
 'https://56367.gradio.app')